In [1]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

sns.set(rc={'figure.figsize': (25, 10)}, font_scale=1.5)

df = pd.read_csv('/content/drive/MyDrive/ECH_2019.csv')

# df = pd.read_csv('ECH_2019.csv')

In [3]:
df.sample(2)



,id_hogar,sexo,edad,ascendencia,parentesco,estado_laboral,nper,sueldo,busca_otro_trabajo,actividad,asistencia alguna vez a enseñanza,edu preescolar,edu primaria,edu media,edu técnica,edu univ o simil,edu terciario no uni,edu posgrado,edu mag prof,barrio,estado_civil,hijos,leer_escribir
30985,2019016698,Mujer,55,Blanca,Esposo/a o compañero/a,Ocupados,2,110000,No,Enseñanza secundaria de formación general,Sí,"Sí, asistió","Sí, asistió","Sí, asistió",No asistió,No asistió,No asistió,"Sí, asistió","Sí, asistió",2,0,Sí,Sí
61786,2019033204,Mujer,39,Blanca,Jefe/a de hogar,Ocupados,1,9000,No,Enseñanza pre primaria y primaria,Sí,"Sí, asistió","Sí, asistió","Sí, asistió",Asiste actualmente,0,0,0,0,25,Soltero/a,No,Sí


In [4]:
no_data = {0: 'No hay dato', '0': 'No hay dato'}

mask = df.columns.difference(['edad', 'sueldo', 'hijos', 'barrio'])

df[mask] = df[mask].replace(no_data)



### AJUSTANDO DATOS PARA ENTRENAR EL MODELO

Seleccionamos personas con edad suficiente para trabajar

In [5]:
df = df[~df['estado_laboral'].str.contains('Menores de 14 años')]

In [6]:
# df = df[df['estado_laboral'].str.contains('Ocupados')]

# Categorizamos las edades y sueldos en rangos

In [7]:
df['rango_sueldos'] = pd.cut(df['sueldo'], bins=[
    1,
    5000,
    10000,
    30000,
    50000,
    100000,
    float('Inf'),
    ]).cat.codes

df['rango_edades'] = pd.cut(df['edad'], bins=[
    0,
    10,
    15,
    20,
    30,
    40,
    50,
    60,
    70,
    float('Inf'),
    ]).cat.codes

Eliminanos las columnas que no vamos a usar o que ya categorizamos

In [8]:
df = df[[
    'sexo', 
    'ascendencia', 
    'parentesco', 
    'estado_laboral', 
    'busca_otro_trabajo',
    'actividad',
    'asistencia alguna vez a enseñanza',
    'edu preescolar',
    'edu primaria',
    'edu media',
    'edu técnica',
    'edu univ o simil',
    'edu terciario no uni',
    'edu posgrado',
    'edu mag prof',
    'barrio',
    'estado_civil',
    'hijos',
    'leer_escribir',
    'rango_sueldos',
    'rango_edades'
    ]]

    

In [9]:
df = df[~df['rango_sueldos'].isna()]

### DEFINIMOS LAS VARIABLES DEPENDIENTES E INDEPENDIENTES

In [10]:
# Feautures
X = df.drop('rango_sueldos', axis=1).to_numpy()
y = df['rango_sueldos']

print('X shape', X.shape, '\nY shape', y.shape)

X shape (49036, 20) 
Y shape (49036,)


In [11]:
from sklearn.preprocessing import OneHotEncoder

one_hot = OneHotEncoder()

X = one_hot.fit_transform(X)


In [ ]:
X

In [ ]:
y

In [12]:
prediction = X.toarray()
prediction = prediction[49000].reshape(1, -1)

### Dividimos los datos en un 30% de test y 70% de entrenamiento

In [13]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42) 



In [14]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)



(34325, 525) (34325,)
(14711, 525) (14711,)


# DEFINIMOS ALGORITMO DE REGRESIÓN LOGISTICA

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

logistic_reg = LogisticRegression(multi_class='multinomial', max_iter=20000)

logistic_reg.fit(X_train, y_train)



LogisticRegression(max_iter=20000, multi_class='multinomial')

### VALOR DE CROSS VALIDATION PARA CADA FOLD

In [16]:
scores = cross_val_score(logistic_reg, X, y, cv=5)

scores



array([0.57432708, 0.57234628, 0.57509942, 0.58091159, 0.57469155])

In [17]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(logistic_reg.score(X_test, y_test),2))
print('Score en train ', round(logistic_reg.score(X_train, y_train),2))



Promedio de cross_validation  0.58
Score en test  0.57
Score en train  0.6


In [18]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

predictions = logistic_reg.predict(X_test)

confusion_matrix(y_test, predictions)



array([[3252,   27,   30, 1068,  105,   52,    8],
       [ 125,   60,   40,  208,    1,    1,    0],
       [ 214,   43,   51,  512,   12,    6,    0],
       [1122,   32,   48, 4000,  440,   46,    2],
       [ 356,    0,    0,  967,  780,  149,    3],
       [ 117,    0,    0,  154,  271,  235,   11],
       [  23,    0,    0,    7,   23,   92,   18]])

### INVESTIGAMOS LA EFECTIVIDAD DEL MODELO SOBRE LOS DATOS

In [19]:
from sklearn.metrics import accuracy_score

#Prediccion en Train
y_train_pred = logistic_reg.predict(X_train)

#Prediccion en Test
y_test_pred = logistic_reg.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Aciertos sobre entrenamiento:', round(train_accuracy, 2))
print('Aciertos sobre evaluación:', round(test_accuracy, 2))



Aciertos sobre entrenamiento: 0.6
Aciertos sobre evaluación: 0.57


In [20]:
y_train

107625   -1
84723     3
61045     4
46237     2
81096    -1
         ..
24931     3
98341     2
84144     3
1898      2
34996    -1
Name: rango_sueldos, Length: 34325, dtype: int8

### APLICAMOS PREDICCIÓN SOBRE EL MODELO

In [21]:
print(logistic_reg.predict(prediction))

print(y_train[49000])



[2]
-1


Determinamos la probabilidad de la predicción

In [22]:
probabilidades_prediccion = logistic_reg.predict_proba(prediction)
probabilidades_prediccion[:,1]



array([0.00061027])

# REPETIMOS EL EJERCICIO CON EL MODELO DE RANDOM FOREST

In [23]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(100)

random_forest.fit(X_train, y_train)



RandomForestClassifier()

In [24]:
scores = cross_val_score(random_forest, X, y, cv=5)

scores



array([0.5262031 , 0.5242174 , 0.52187213, 0.52431936, 0.52564495])

In [25]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(logistic_reg.score(X_test, y_test),2))
print('Score en train ', round(logistic_reg.score(X_train, y_train),2))



Promedio de cross_validation  0.52
Score en test  0.57
Score en train  0.6


In [26]:
predictions = logistic_reg.predict(X_test)

confusion_matrix(y_test, predictions)



array([[3252,   27,   30, 1068,  105,   52,    8],
       [ 125,   60,   40,  208,    1,    1,    0],
       [ 214,   43,   51,  512,   12,    6,    0],
       [1122,   32,   48, 4000,  440,   46,    2],
       [ 356,    0,    0,  967,  780,  149,    3],
       [ 117,    0,    0,  154,  271,  235,   11],
       [  23,    0,    0,    7,   23,   92,   18]])

In [27]:
#Prediccion en Train
y_train_pred = random_forest.predict(X_train)

#Prediccion en Test
y_test_pred = random_forest.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Aciertos sobre entrenamiento:', round(train_accuracy, 2))
print('Aciertos sobre evaluación:', round(test_accuracy, 2))



Aciertos sobre entrenamiento: 0.94
Aciertos sobre evaluación: 0.52


In [28]:
print(random_forest.predict(prediction))
print(y_train[49000])



[3]
-1


# REPETIMOS EL EJERCICIO CON EL MODELO DE K NEAREST NEIGHBOURS

In [29]:
from sklearn.neighbors import KNeighborsClassifier

knearest_model = KNeighborsClassifier(n_neighbors=4)

knearest_model.fit(X_train, y_train)



KNeighborsClassifier(n_neighbors=4)

In [30]:
scores = cross_val_score(knearest_model, X, y, cv=5)

scores           



array([0.45778956, 0.45426736, 0.44763944, 0.4442745 , 0.44702763])

In [31]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(knearest_model.score(X_test, y_test),2))
print('Score en train ', round(knearest_model.score(X_train, y_train),2))



Promedio de cross_validation  0.45
Score en test  0.45
Score en train  0.64


In [32]:
predictions = knearest_model.predict(X_test)

confusion_matrix(y_test, predictions)



array([[2767,   62,   84, 1250,  283,   89,    7],
       [ 149,   58,   62,  157,    8,    1,    0],
       [ 291,   75,  104,  334,   28,    5,    1],
       [1770,   95,  228, 3010,  515,   72,    0],
       [ 722,    6,   19,  882,  497,  119,   10],
       [ 263,    1,    7,  163,  185,  153,   16],
       [  61,    0,    0,   17,   22,   49,   14]])

In [33]:
#Prediccion en Train
y_train_pred = knearest_model.predict(X_train)

#Prediccion en Test
y_test_pred = knearest_model.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Aciertos sobre entrenamiento:', round(train_accuracy, 2))
print('Aciertos sobre evaluación:', round(test_accuracy, 2))



Aciertos sobre entrenamiento: 0.64
Aciertos sobre evaluación: 0.45


In [34]:
print(knearest_model.predict(prediction))
print(y_train[49000])



[-1]
-1
